# 引数に対してイテレータを使うときには確実さを優先する

In [26]:
# 旅行者の人数について分析
# 各都市への訪問者数のデータセットから、旅行者全体の何パーセントを各都市で受け入れているかを計算
def normalize(numbers):
  print(f'numbers: {numbers}')
  total = sum(numbers)
  print(f'total: {total}')
  result = []
  for value in numbers:
    print(f'value: {value}')
    percent = 100 * value / total
    result.append(percent)
  print(f'result: {result}')
  return result

In [4]:
visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [ ]:
# スケールアップのため
# すべての都市を含んだファイルからデータを読み込むように
# 全世界の旅行者数のようなずっと大きなもっとメモリが必要なデータセットにも同じ関数を利用したいので、
# ジェネレータを定義
def read_visits(data_path):
  with open(data_path) as f:
    for line in f:
      yield int(line)

In [27]:
# ジェネレータ read_visits の戻り値に normalize を呼び出しても何も結果が得られない
# 原因は、イテレータが結果を一度しか生成しないため
# StopIteration 例外をすでに起こしたイテレータやジェネレータに反復処理をしても何の結果も得られない
it = read_visits('my_numbers.txt')
percentages = normalize(it) # normalize の sum(numbers) で反復処理が完了したため、その後の numbers で [] を返すようになった
print(percentages)

numbers: <generator object read_visits at 0x7f7d06f2a200>
total: 130
result: []
[]


In [ ]:
it = read_visits('my_numbers.txt')
print(list(it))
print(list(it)) # すでに反復処理が完了

[15, 35, 80]
[]


#### 問題点
困るのは、すでに完了したイテレータに対して反復処理をしても、何のエラーも生じないこと

for ループ、list コンストラクタ、Python 標準ライブラリの他の多くの関数では、通常の演算においては StopIteration 例外が発生することを想定している。
これらの関数では、出力を持たないイテレータと、出力はあったが終了したイテレータとを区別することができない

In [30]:
# 解決策
def normalize_copy(numbers):
  numbers = list(numbers) # 内容全体の複製をリストに保持
  total = sum(numbers)
  result = []
  for value in numbers:
    percent = 100 * value / total
    result.append(percent)
  return result

In [32]:
it = read_visits('my_numbers.txt')
percentages = normalize_copy(it)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


ただし、これにも問題がある

入力イテレータの複製の内容が巨大になりうるという問題

In [33]:
# 解決策

# 呼ばれるたびに新たなイテレータを返す関数を受け入れる
def normalize_func(get_iter):
  total = sum(get_iter()) # 新たなイテレータ
  result = []
  for value in get_iter(): # 新たなイテレータ
    percent = 100 * value / total
    result.append(percent)
  return result

In [43]:
path = 'my_numbers.txt'
percentages = normalize_func(lambda: read_visits(path))
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


これは動作するが、Lambda 関数を渡さなければならないのは面倒

In [46]:
# 解決策

# イテレータプロトコルを実装した新たなコンテナクラスを提供
# イテレータプロトコル :
# Python の for ループや関連する式が、コンテナ型の内容をどのように横断するかを示すもの
class ReadVisits:
  def __init__(self, data_path):
    self.data_path = data_path

  # 自分のクラスに対するこれらの振る舞いすべてをジェネレータとした __iter__ メソッドを実装
  def __iter__(self):
    with open(self.data_path) as f:
      for line in f:
        yield int(line)

In [ ]:
# 動作する

# 動作するのは、normalize の sum メソッドが新たなイテレータオブジェクトを生成するために
# ReadVisits.__iter__ を呼び出すから
# これらのイテレータは、それぞれ独立に終わるまで進められれ、
# どの反復処理でも入力データ値がすべて処理されることを保証
visits = ReadVisits(path)
percentages = normalize(visits)
print(percentages)
assert sum(percentages) == 100.0

[11.538461538461538, 26.923076923076923, 61.53846153846154]


この方式には、唯一、入力データを複数回読み込んでしまうという欠点がある

イテレータプロトコルでは、組み込み関数 iter にイテレータが渡されると、iter がイテレータそのものを返すことになっている。
対照的に、コンテナ型が iter に渡されると、そのたびに、新たなイテレータオブジェクトが返される。
そうしてこの振る舞いの入力値をテストして、条件を満たさないと、TypeError を起こして反復できない引数を拒絶する。

In [49]:
def normalize_defensive(numbers):
  # 反復できない引数を拒絶
  # iter(numbers) is numbers が True であれば、numbers がイテレータであることがわかる
  # つまり、そのままイテレータで進めると予期せぬ失敗になる
  # イテレータは1度使うとそれきりなので、2度目には[]が返ってくるので、それを防ぐための if 文
  if iter(numbers) is numbers:
    raise TypeError('Must supply a container')
  total = sum(numbers)
  result = []
  for value in numbers:
    percent = 100 * value / total
    result.append(percent)
  return result

In [50]:
from collections.abc import Iterator

def normalize_defensive(numbers):
  # 組み込みモジュールを使った別のやりかた
  if isinstance(numbers, Iterator):
    raise TypeError('Must sypply a container')
  total = sum(numbers)
  result = []
  for value in numbers:
    percent = 100 * value / total
    result.append(percent)
  return result

In [52]:
visits = [15, 35, 80]
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

visits = ReadVisits(path)
percentages = normalize_defensive(visits)
assert sum(percentages) == 100.0

In [ ]:
visits = [15, 35, 80]
it = iter(visits)
percentages = normalize_defensive(it) # イテレータの時は、期待通り TypeError がスローされる

TypeError: Must sypply a container